In [3]:
# Import necessary libraries
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import json
import plotly.graph_objects as go

In [5]:
url  = [ #
    #ibra50
    'https://raw.githubusercontent.com/BDonadelli/Finance-playground/main/data/Cart_IBr50.csv',
    #ibra100
    'https://raw.githubusercontent.com/BDonadelli/Finance-playground/main/data/Cart_IBr100.csv',
    #div
    'https://raw.githubusercontent.com/BDonadelli/Finance-playground/main/data/Cart_Idiv.csv' ,
    #small
    'https://raw.githubusercontent.com/BDonadelli/Finance-playground/main/data/Cart_Small.csv',
    #ibbr
    'https://raw.githubusercontent.com/BDonadelli/Finance-playground/main/data/Cart_IBBR.csv',
    #low vol
    'https://raw.githubusercontent.com/BDonadelli/Finance-playground/main/data/Cart_Ibov_LowVol.csv'
]

escolha = 0 #int(input('0-ibra50; 1-ibra100; 2-idiv; 3-small; 4- ibbr ; 5-low vol: '))
indice = { 0:'Ibra50' , 1:'Ibra100' , 2:'Idiv' , 3:'Small' , 4:'IBBr' , 5:'Low Vol'}

In [15]:
import os
from urllib.parse import urlparse

arquivo = os.path.basename(urlparse(url[escolha]).path) 
# Caminho do arquivo local
file_path = os.path.expanduser(f'~/GHub/Codigos-em-financas/data/{arquivo}')

# Verificar se o arquivo existe localmente
if os.path.exists(file_path):
    # Ler o arquivo local
    ticker_list = pd.read_csv( file_path,sep=';' , decimal=',' , thousands='.' , 
                header=0 , skiprows=0 , skipfooter=2 , 
                encoding='utf-8',index_col=False , engine='python')['Código'].to_list()
    print("Arquivo lido localmente.")
else:
    # Ler o arquivo a partir da URL
    ticker_list = pd.read_csv(url[escolha] ,sep=';' , decimal=',' , thousands='.' , 
                header=1 , skiprows=0 , skipfooter=2 , 
                encoding='latin-1',index_col=False , engine='python')['Código'].to_list()
    print("Arquivo lido da URL.")

Arquivo lido localmente.


In [16]:
years=2
end_date = datetime.now()
start_date = end_date - timedelta(days=years * 365)
stock_data = pd.DataFrame()

for ticker in ticker_list:
    stock = yf.Ticker(ticker+'.SA')
    hist_data = stock.history(period='1d', start=start_date, end=end_date)
    close_data = hist_data['Close'].rename(ticker)
    stock_data = pd.merge(stock_data, pd.DataFrame(close_data), left_index=True, right_index=True, how='outer')

In [22]:
daily_data = stock_data.copy()
daily_data

,VALE3,ITUB4,PETR4,PETR3,WEGE3,ELET3,BBAS3,BBDC4,SBSP3,B3SA3,...,ALOS3,ASAI3,HYPE3,BRAV3,CSNA3,MULT3,AZZA3,CYRE3,MGLU3,USIM5
Date,,,,,,,,,,,,,,,,,,,,,
2022-09-26 00:00:00-03:00,59.482273,24.384430,18.626062,20.272074,29.589149,43.767960,16.808010,17.423641,42.093910,11.946054,...,NaN,17.634159,42.128613,NaN,9.467435,22.000753,92.091339,14.975771,41.902485,6.693896
2022-09-27 00:00:00-03:00,58.599392,24.243532,18.867712,20.515491,29.085835,42.509865,16.596994,17.317724,41.749973,11.834672,...,NaN,17.673922,41.709751,NaN,9.298374,21.862673,91.834389,14.864769,39.096516,6.638574
2022-09-28 00:00:00-03:00,59.153355,24.234722,18.613348,20.371937,29.347166,41.910305,16.601295,17.326546,44.033325,11.927492,...,NaN,17.842907,41.253651,NaN,9.382906,22.020947,92.072304,16.076519,40.499500,6.629354
2022-09-29 00:00:00-03:00,59.231262,24.595783,18.638783,20.390661,29.172947,41.743217,16.510862,17.458948,44.329491,11.695440,...,NaN,17.842907,40.914028,NaN,9.252266,22.076578,91.786812,15.956271,37.880596,6.454169
2022-09-30 00:00:00-03:00,62.355984,24.710258,18.950378,20.646563,31.108768,42.234657,16.588375,17.511906,47.386696,12.113132,...,NaN,17.445292,41.476418,NaN,9.774820,22.410366,93.652054,16.973772,41.902485,6.933622
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-09-19 00:00:00-03:00,58.230000,36.327019,36.270000,39.700001,53.259998,41.200001,27.910000,15.030000,94.970001,11.898141,...,22.90,8.240000,28.076155,18.889999,12.050000,26.540001,46.299999,21.830000,11.160000,6.230000
2024-09-20 00:00:00-03:00,57.349998,35.930000,36.259998,39.700001,52.520000,39.930000,27.500000,14.750000,94.660004,11.501536,...,22.35,7.700000,27.113146,18.180000,11.130000,26.340000,44.650002,20.730000,10.350000,5.880000
2024-09-23 00:00:00-03:00,57.529999,35.720001,36.630001,40.200001,54.320000,39.549999,27.400000,14.370000,93.620003,11.104932,...,22.18,7.860000,26.904661,17.900000,11.080000,26.120001,45.200001,20.190001,10.020000,5.600000


In [23]:
def resample_data(data, period):
    if period == 'D':
        return data
    elif period == 'W':
        return data.resample('W').last()
    elif period == 'M':
        return data.resample('M').last()

In [24]:
def simulate_momentum_strategy(data, initial_amount, top_n, tax_rate = 0.15 , period='M'):
    data = resample_data(data, period)
    log_returns = np.log(data / data.shift(1))
    simulation_details = pd.DataFrame(index=log_returns.index,
                                      columns=['Portfolio', 'Profit Before Tax', 'Tax', 'Portfolio Value'])
    cash = initial_amount

    # strategy
    for i in range(0, len(log_returns) - 1):
        # top n momentum 
        top_stocks = log_returns.iloc[i].sort_values(ascending=False).head(top_n)
        top_stocks = top_stocks[top_stocks > 0]

        if not top_stocks.empty:
            simulation_details.loc[log_returns.index[i + 1], 'Portfolio'] = json.dumps(top_stocks.index.tolist())
            num_stocks = len(top_stocks)
            allocation_per_stock = cash / num_stocks
            # Calculo do valor do novo portfólio com base nos retornos do dia seguinte
            new_value = sum(allocation_per_stock * np.exp(log_returns.loc[log_returns.index[i + 1], stock]) for stock in top_stocks.index)
            #  Calcular e deduzir o imposto se houver lucro
            profit = new_value - cash
            simulation_details.loc[log_returns.index[i + 1], 'Profit Before Tax'] = round(profit, 2)

            if profit > 0:
                tax = profit * tax_rate
                new_value -= tax
                simulation_details.loc[log_returns.index[i + 1], 'Tax'] = round(tax, 2)
            simulation_details.loc[log_returns.index[i + 1], 'Portfolio Value'] = round(new_value, 2)

        else:
            ## Sem alocação, então o valor do portfólio permanece o mesmo
            simulation_details.loc[log_returns.index[i + 1], 'Portfolio Value'] = cash
        # Atualizar valor em dinheiro para a próxima rodada
        cash = simulation_details.loc[log_returns.index[i + 1], 'Portfolio Value']
    # Atribuir o valor inicial à primeira linha
    simulation_details.loc[log_returns.index[0], 'Portfolio Value'] = initial_amount
    return simulation_details

In [29]:
initial_amount = 100
top_n = 3
frequency='M'
tax_rate=.15
simulation_details = simulate_momentum_strategy(daily_data, initial_amount, top_n,tax_rate,frequency)
simulation_details

,Portfolio,Profit Before Tax,Tax,Portfolio Value
Date,,,,
2022-09-30 00:00:00-03:00,NaN,NaN,NaN,100
2022-10-31 00:00:00-03:00,NaN,NaN,NaN,100
2022-11-30 00:00:00-03:00,"[""PRIO3"", ""WEGE3"", ""SBSP3""]",-0.33,NaN,99.67
2022-12-31 00:00:00-03:00,"[""GGBR4"", ""CSNA3"", ""VALE3""]",0.33,0.05,99.95
2023-01-31 00:00:00-03:00,"[""BBSE3"", ""EMBR3"", ""VALE3""]",10.88,1.63,109.2
2023-02-28 00:00:00-03:00,"[""MGLU3"", ""CSNA3"", ""NTCO3""]",-8.58,NaN,100.62
2023-03-31 00:00:00-03:00,"[""MULT3"", ""NTCO3"", ""TIMS3""]",-4.3,NaN,96.32
2023-04-30 00:00:00-03:00,"[""EMBR3"", ""CMIG4"", ""RADL3""]",3.29,0.49,99.12
2023-05-31 00:00:00-03:00,"[""BPAC11"", ""CPLE6"", ""TIMS3""]",1.5,0.22,100.39


In [30]:
# Simula o desempenho de cada ação individual no mesmo período
def track_individual_investments(data, initial_amount, simulation_details, period='W'):
    # Reamostrar dados com base no período especificado e calcular retornos 
    data = resample_data(data, period)
    returns = data.pct_change()
    # armazenar valores de ações individuais ao longo do tempo
    individual = pd.DataFrame(index=data.index, columns=data.columns)
    for stock in data.columns:
        # Simular um investimento em cada ação
        individual[stock] = (1 + returns[stock]).cumprod() * initial_amount
    # Incluir o Valor do Portfólio da estratégia de momentum
    individual['Portfolio Value'] = simulation_details['Portfolio Value']
    individual['Baseline'] = individual.iloc[:, :-1].T.mean()
    # Ajustar os primeiros valores para corresponder ao Valor Inicial.
    individual.iloc[0, :] = initial_amount
    return individual.fillna(0).astype(int)

individual_investments_df = track_individual_investments(daily_data, initial_amount, simulation_details, frequency)

individual_investments_df

,VALE3,ITUB4,PETR4,PETR3,WEGE3,ELET3,BBAS3,BBDC4,SBSP3,B3SA3,...,HYPE3,BRAV3,CSNA3,MULT3,AZZA3,CYRE3,MGLU3,USIM5,Portfolio Value,Baseline
Date,,,,,,,,,,,,,,,,,,,,,
2022-09-30 00:00:00-03:00,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,100
2022-10-31 00:00:00-03:00,93,108,100,100,125,115,96,100,121,115,...,114,0,96,109,106,101,99,96,100,108
2022-11-30 00:00:00-03:00,118,92,95,103,121,110,93,78,121,97,...,100,0,123,94,91,78,76,103,99,102
2022-12-31 00:00:00-03:00,123,91,87,95,119,98,92,76,115,101,...,102,0,128,91,80,71,61,95,99,99
2023-01-31 00:00:00-03:00,131,92,93,100,119,94,108,73,112,99,...,105,0,163,99,89,82,98,113,109,105
2023-02-28 00:00:00-03:00,118,92,90,97,122,79,109,68,104,81,...,94,0,147,105,76,81,80,90,100,97
2023-03-31 00:00:00-03:00,114,91,84,90,127,77,107,69,102,80,...,86,0,136,103,65,76,73,96,96,95
2023-04-30 00:00:00-03:00,103,95,95,100,129,79,117,73,93,90,...,85,0,125,109,68,84,74,100,99,97
2023-05-31 00:00:00-03:00,90,97,105,110,118,82,122,82,108,103,...,93,0,112,111,75,105,84,97,100,102


In [31]:
from scipy.stats import ttest_1samp

def sharpe_ratio(returns, annual_risk_free_rate=0.11, frequency='D'):
    if frequency == 'D':
        adjusted_rfr = (1 + annual_risk_free_rate) ** (1/252) - 1
    elif frequency == 'W':
        adjusted_rfr = (1 + annual_risk_free_rate) ** (1/52) - 1
    elif frequency == 'M':
        adjusted_rfr = (1 + annual_risk_free_rate) ** (1/12) - 1

    excess_returns = returns - adjusted_rfr
    return excess_returns.mean() / excess_returns.std()

def t_test_portfolio_returns(portfolio_returns, bench_annual_rate=0.1, frequency='D'):
    if frequency == 'D':
        adjusted_rfr = (1 + bench_annual_rate) ** (1/252) - 1
    elif frequency == 'W':
        adjusted_rfr = (1 + bench_annual_rate) ** (1/52) - 1
    elif frequency == 'M':
        adjusted_rfr = (1 + bench_annual_rate) ** (1/12) - 1

    t_stat, p_value = ttest_1samp(portfolio_returns[1:], adjusted_rfr)  # [1:] to exclude the NaN from pct_change
    return t_stat, p_value

def calculate_metrics(dataframe, initial_amount, bench_annual_rate, frequency='D'):
    # Calculate the final and relative values
    final_values = dataframe.iloc[-1]
    relative_values = final_values / initial_amount - 1  # Subtract 1 to get the growth proportion

    # Calculate mean return and Sharpe Ratio
    returns = dataframe.pct_change()

    if frequency == 'D':
        annualization_factor = 252
    elif frequency == 'W':
        annualization_factor = 52
    elif frequency == 'M':
        annualization_factor = 12

    # Corrected annualization of mean returns
    mean_returns = (1 + returns.mean()) ** annualization_factor - 1
    sharpes = returns.apply(calculate_sharpe_ratio, annual_risk_free_rate=0.01, frequency=frequency)

    # Test if the portfolio returns are greater than the adjusted risk-free rate
    portfolio_returns = dataframe['Portfolio Value'].pct_change()
    t_stat, p_value = t_test_portfolio_returns(portfolio_returns, bench_annual_rate, frequency=frequency)

    return final_values, relative_values, mean_returns, sharpes, t_stat, p_value / 2

bench_annual_rate = 0.1

# Calculate the metrics
final_values, relative_values, mean_returns, sharpes, t_stat, p_value = calculate_metrics(individual_investments_df, initial_amount, bench_annual_rate, frequency)

/home/yair/bin/miniconda3/lib/python3.11/site-packages/pandas/core/nanops.py:1010: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


In [41]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def plot_combined_charts(dataframe, final_values, relative_values, sharpes, mean_returns):
    labels = final_values.index
    colors = ['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A']

    fig = make_subplots(rows=3, cols=2,
                        subplot_titles=('Portfolio Value Over Time',
                                        '',
                                        'Final Investment Values',
                                        'Relative Investment Growth',
                                        'Annualized Sharpe Ratios',
                                        'Annualized Mean Returns'),
                        vertical_spacing=0.08)

    # Portfolio Value line chart
    fig.add_trace(go.Scatter(x=dataframe.index,
                             y=dataframe['Portfolio Value'],
                             mode='lines',
                             name='Portfolio Value',
                             line=dict(color=colors[0], width=2.5)),
                  row=1, col=1)

    # T-test and P-value
    significance_text = f"<b>T-test:</b> {t_stat:.2f}<br><b>P-value:</b> {p_value:.5f}"
    if t_stat > 2 and p_value < 0.05:
        significance_text += f"<br><b>Significantly different from {bench_annual_rate:.0%} per year!</b>"

    fig.add_annotation(
        text=significance_text,
        showarrow=False,
        xref="x2", yref="y2",
        x=0.5, y=0.5,
        font=dict(size=15),
        bgcolor="white",
        align="center"
    )

    # Final values
    fig.add_trace(go.Bar(x=labels,
                         y=final_values.values,
                         name='Final Values ($)',
                         text=[f"${v:,.2f}" for v in final_values.values],
                         textposition='outside',
                         marker_color=colors[1]),
                  row=2, col=1)

    # Relative Growth
    fig.add_trace(go.Bar(x=labels,
                         y=relative_values.values,
                         name='Relative Growth',
                         text=[f"{v:.2%}" for v in relative_values.values],
                         textposition='outside',
                         marker_color=colors[2]),
                  row=2, col=2)

    # Sharpe Ratios
    fig.add_trace(go.Bar(x=labels,
                         y=sharpes.values,
                         name='Annualized Sharpe Ratio',
                         text=[f"{v:.2f}" for v in sharpes.values],
                         textposition='outside',
                         marker_color=colors[3]),
                  row=3, col=1)

    # Mean Returns
    fig.add_trace(go.Bar(x=labels,
                         y=mean_returns.values,
                         name='Annualized Mean Returns',
                         text=[f"{v:.2%}" for v in mean_returns.values],
                         textposition='outside',
                         marker_color=colors[4]),
                  row=3, col=2)

    # Update layout
    fig.update_layout(title_text="Investment Results Overview",
                      title_font=dict(size=24, color='black', family="Arial Black"),
                      title_pad=dict(t=10),
                      showlegend=False,
                      height=1500,
                      title_x=0.5,
                      bargap=0.05,
                      )

    fig.show()

plot_combined_charts(individual_investments_df, final_values, relative_values, sharpes, mean_returns)

In [ ]:
from scipy.optimize import linprog
import numpy as np

# Definir a matriz de custo C (forma 1D)
C = np.array([1,1,1,1])

# Definir os vetores de probabilidade
a = np.array([1/3,2/3])
b = np.array([1/4,3/4])

# Restrições de igualdade para linhas (somatório das linhas = massa a)
A_eq = np.zeros((len(a), len(a) * len(b)))
for i in range(len(a)):
    A_eq[i, i*len(b):(i+1)*len(b)] = 1

# Restrições de igualdade para colunas (somatório das colunas = massa b)
B_eq = np.zeros((len(b), len(a) * len(b)))
for j in range(len(b)):
    B_eq[j, j::len(b)] = 1

# Combinar as restrições
A_eq = np.vstack([A_eq, B_eq])
b_eq = np.hstack([a, b])

# Limites para as variáveis (todas >= 0)
bounds = [(0, None)] * len(C)

# Resolver o problema
result = linprog(C, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method='highs')

# Plano de transporte ótimo
P = result.x.reshape((len(a), len(b)))
print(P)
